Data Cleaning to analyze given data set having relationship between lifestyle behaviors (activity, sleep, diet) and glucose regulation in diabetic patients.

In [643]:
import glob
import os
import pandas as pd
from datetime import timedelta

In [644]:
# Get all csv files from folder
files = glob.glob("/Users/venmeen/Downloads/HUPA-UC Diabetes Dataset/*.csv")

# Reading demographic patient file
demographic_df = pd.read_csv("/Users/venmeen/Downloads/HUPA-UC Diabetes Dataset/T1DM_patient_sleep_demographics_with_race.csv")

<h4>Merging 25 patient data files as one file </h4>
<h5><font face="TimesNewRoman">Adding patient_Id column as unique identifier based on filename</h5>
<h5><font face="TimesNewRoman">Reasoning: Merging files together will help for group analysis like finiding glucose trends and other general pattern across patients</h5>

In [646]:
def merge_rawfiles():
# Merging raw data file as one file
    df_files = []
    for file in files:

        filename = os.path.splitext(os.path.basename(file))[0]
        if not filename.startswith("HUPA"):
            continue
        df = pd.read_csv(file,sep=";") 

        # Uniform column header in all files
        df.columns = df.columns.str.strip().str.lower()

        # Removing duplicate rows - if more than one record has the same time for single patient, then it is considered as duplicate.
        df.drop_duplicates(subset=["time"], inplace= True)

        # Make sure time as Date time
        df['time'] = pd.to_datetime(df['time'], errors='coerce')

        # check if patient has more than 15 days of data
        max_date = df['time'].max()
        min_date = df['time'].min()
        total_days = (max_date - min_date).days
        total_days = df['time'].dt.date.nunique()
        
        if total_days > 15:
            # filter only last 15 days
            cutoff = max_date - timedelta(days=15)
            df = df[df['time'] > cutoff]
    
        #Add patient_id since we are merging all files together
        df["patient_id"] = filename
        df_files.append(df)

    # Merge the patient data files
    df = pd.concat(df_files, ignore_index=True)

    # Saving it as Single Merged csv file
    df.to_csv("mergedraw_file.csv", index=False)
    return df

<h4>Verying data by checking for column's data types,null values and Nan values</h4>

In [648]:
def verify_data():
    print("\033[1mDataFrame's Information:\033[0m\n")
    print(df.info())
    print("\033[0m\nNull Value Count:\033[0m\n",df.isnull().sum())
    print("\033[0m\nNan Values:\033[0m\n",df.isna().sum())
    print("\033[0m\nNumber of rows and cols:\033[0m\n",df.shape)
    print("\033[0m\nDescription of DataFrame :\033[0m\n")
    print(df.describe)

<h4>Display Raw Merged File Information as it is</h4>

In [650]:
df = merge_rawfiles()
print("\033[1mRaw Merged Data Info:\033[0m\n")
verify_data()

Raw Merged Data Info:

DataFrame's Information:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90539 entries, 0 to 90538
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   time                    90539 non-null  datetime64[ns]
 1   glucose                 90539 non-null  float64       
 2   calories                90539 non-null  float64       
 3   heart_rate              90539 non-null  float64       
 4   steps                   90539 non-null  float64       
 5   basal_rate              90539 non-null  float64       
 6   bolus_volume_delivered  90539 non-null  float64       
 7   carb_input              90539 non-null  float64       
 8   patient_id              90539 non-null  object        
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 6.2+ MB
None

Null Value Count:
 time                      0
glucose                   0
calories                  0
hea

<h4>Setting index with time and patient_id</h4>
<h5><font face="TimesNewRoman">set time and patient_id as multi_index to identify particular record.</font></h5>
<h5><font face="TimesNewRoman">Reasoning: This index will help for resampling, rolling averages and for plotting</font></h5>

In [652]:
def set_index(df):

    # Setting the time and patient_id column as multi_index 
    df.set_index(['time','patient_id'],inplace=True)

<h4> Ensure the columns DataTypes</h4>

In [654]:
def ensure_col_dtype():

    # Collecting column names in one list
    cols_dtype = ['glucose', 'calories', 'heart_rate','steps','basal_rate','bolus_volume_delivered','carb_input']

    #Ensure all the columns in list are in numeric
    df[cols_dtype] = df[cols_dtype].apply(pd.to_numeric, errors='coerce')

    #Setting patient_id column as string
    df['patient_id'] = df['patient_id'].astype("string")

    #Setting time column as datetime
    df["time"] = pd.to_datetime(df["time"],errors="coerce")
    return df

<h4> Standardize numeric column values </h4>
<font face="TimesNewRoman">Rounding the float value to 3 decimal numbers for clarity and usability</font>

In [656]:
def standardize_numeric_cols():
    stdardize_cols = ['glucose', 'calories', 'heart_rate']
    df[stdardize_cols] = df[stdardize_cols].round(2)
    return df

<h4> Check for negative values in numeric columns </h4>
<font face="TimesNewRoman">To treat the negative values as error</font>

In [658]:
def check_negative_values():
    columns = ['glucose', 'calories', 'heart_rate', 'steps', 'basal_rate', 'bolus_volume_delivered', 'carb_input']
    col_negative_values = (df[columns] < 0).any()
    print(col_negative_values)

<h4> Display After Treating the Data</h4>

In [660]:
print("Column List:\n")
print(df.columns.tolist())
df = standardize_numeric_cols()
df = ensure_col_dtype()
#set_index(df)
#print("\nIndexes:\n")
#print(df.index.names)
print("\033[1mChecking for negative values in columns:\033[0m\n")
check_negative_values()

Column List:

['time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate', 'bolus_volume_delivered', 'carb_input', 'patient_id']
Checking for negative values in columns:

glucose                   False
calories                  False
heart_rate                False
steps                     False
basal_rate                False
bolus_volume_delivered     True
carb_input                False
dtype: bool


<h4>Setting up glucose range</h4>
Glucose range provides framework to evaluate a patient's blood sugar control
Considering glucose level from 40-500 as valid data sice the libre device dont record beyond this.

In [662]:
def classify_by_glucose_value(value):
    if pd.isna(value): 
        return "NA"
    if value < 40 or value > 500:
        return "Invalid"
    elif value < 70:
        return "Below Range"
    elif value > 180:
        return "Above Range"
    else:
        return "In Range"
        
def set_glucose_range(df):
    df["glucose_range_level"] = df["glucose"].apply(classify_by_glucose_value)

<h4>Setting up Calories Burned into categories for easy Analysis</h4>
<h4><font face="TimeNewRoman">Classifying the burned calories data as (0-4 calories burned)'Resting', (5-19 calories burned)'Light Activity', (20-34 calories burned)'Moderate Activity', (35-50 calories burned)'Intense activity', (50-59 calories burned)'Very Extreme/SPIKE' and more then 60 is considered as Error since this is 5 min interval data</font></h4>

In [664]:
def set_caloriesburned_categories(value):
    if value < 5:
        return "Resting"
    elif 5 <= value <20:
        return "Light Activity"
    elif 20 <= value <35:
        return "Moderate Activity"
    elif 35 <= value <50:
        return "Intense activity"
    elif 50 <= value <60:
        return "Very Extreme/SPIKE"
    elif value >60:
        return "ERROR"
def calories_categories(df):
    df['calories_categories'] = df['calories'].apply(set_caloriesburned_categories)

In [665]:
set_glucose_range(df)
calories_categories(df)

<h4>Sorting the data by index</h4>

In [667]:
#df.sort_index(ascending=True,inplace= True)
#df

<h4>Display After setting Range validation columns</h4>

In [669]:
print("\033[1mAfter DataCleanup:\033[0m\n")
verify_data()

After DataCleanup:

DataFrame's Information:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90539 entries, 0 to 90538
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   time                    90539 non-null  datetime64[ns]
 1   glucose                 90539 non-null  float64       
 2   calories                90539 non-null  float64       
 3   heart_rate              90539 non-null  float64       
 4   steps                   90539 non-null  float64       
 5   basal_rate              90539 non-null  float64       
 6   bolus_volume_delivered  90539 non-null  float64       
 7   carb_input              90539 non-null  float64       
 8   patient_id              90539 non-null  string        
 9   glucose_range_level     90539 non-null  object        
 10  calories_categories     90539 non-null  object        
dtypes: datetime64[ns](1), float64(7), object(2), string(1)
memor

In [670]:
# Saving it as Single Merged csv file
#df.to_csv("mergedclean_file.csv", index=False)

In [671]:
# Replace with the patient_id you want
patient = "HUPA0027P"
patient27_file = pd.read_csv("/Users/venmeen/Downloads/HUPA-UC Diabetes Dataset/HUPA0027P.csv",sep=';')
print(patient27_file.columns.tolist())
patient27_file['time'] = patient27_file['time'].astype(str).str.strip()
patient27_file["time"] = pd.to_datetime(patient27_file["time"],errors="coerce")
patient27_file.drop_duplicates(subset=["time"], inplace= True)
print("Print Duplicates:\n")
patient27_file.duplicated().sum()
print(patient27_file['time'].dtype)
print(patient27_file.shape)

['time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate', 'bolus_volume_delivered', 'carb_input']
Print Duplicates:

datetime64[ns]
(165306, 8)


In [672]:
patient27_file.duplicated().sum()

0

In [673]:
# Make sure 'time' column is datetime type
df['time'] = pd.to_datetime(df['time'])

# Group by patient and find first and last timestamp
time_range = df.groupby('patient_id')['time'].agg(['min', 'max',]).reset_index()

# Optional: Rename columns
time_range.columns = ['patient_id', 'first_time', 'last_time']

# Display result
print(time_range)

record_counts = df.groupby('patient_id').size().reset_index(name='record_count')
print(record_counts)

   patient_id          first_time           last_time
0   HUPA0001P 2018-06-13 18:40:00 2018-06-27 23:55:00
1   HUPA0002P 2018-06-13 22:45:00 2018-06-24 23:45:00
2   HUPA0003P 2018-06-13 21:40:00 2018-06-26 23:45:00
3   HUPA0004P 2018-07-09 15:30:00 2018-07-20 16:45:00
4   HUPA0005P 2018-07-09 14:20:00 2018-07-22 23:45:00
5   HUPA0006P 2018-07-09 15:45:00 2018-07-17 14:30:00
6   HUPA0007P 2018-09-19 14:25:00 2018-10-02 23:45:00
7   HUPA0009P 2018-09-19 14:35:00 2018-10-02 20:10:00
8   HUPA0010P 2018-11-07 15:35:00 2018-11-17 23:30:00
9   HUPA0011P 2018-11-07 15:55:00 2018-11-20 23:45:00
10  HUPA0014P 2018-11-07 16:45:00 2018-11-20 23:45:00
11  HUPA0015P 2019-03-27 16:15:00 2019-04-09 20:10:00
12  HUPA0016P 2019-03-27 16:15:00 2019-04-09 23:45:00
13  HUPA0017P 2019-03-28 12:00:00 2019-04-09 23:50:00
14  HUPA0018P 2019-07-03 11:15:00 2019-07-16 23:45:00
15  HUPA0019P 2019-07-03 14:00:00 2019-07-16 11:10:00
16  HUPA0020P 2019-07-03 20:15:00 2019-07-13 18:40:00
17  HUPA0021P 2019-07-03 12:

In [674]:
# Ensure time column is datetime
df['time'] = pd.to_datetime(df['time'])

# Calculate number of days per patient
df['date'] = df['time'].dt.date
days_per_patient = df.groupby('patient_id')['date'].nunique().reset_index()
days_per_patient.columns = ['patient_id', 'num_days']

# Count total records per patient
records_per_patient = df.groupby('patient_id')['time'].count().reset_index()
records_per_patient.columns = ['patient_id', 'record_count']

# Merge info
patient_info = pd.merge(days_per_patient, records_per_patient, on='patient_id')
print(patient_info)

   patient_id  num_days  record_count
0   HUPA0001P        15          4096
1   HUPA0002P        12          3181
2   HUPA0003P        14          3770
3   HUPA0004P        12          3184
4   HUPA0005P        14          3858
5   HUPA0006P         9          2290
6   HUPA0007P        14          3857
7   HUPA0009P        14          3812
8   HUPA0010P        11          2976
9   HUPA0011P        14          3839
10  HUPA0014P        14          3829
11  HUPA0015P        14          3792
12  HUPA0016P        14          3835
13  HUPA0017P        13          3599
14  HUPA0018P        14          3895
15  HUPA0019P        14          3711
16  HUPA0020P        11          2862
17  HUPA0021P         9          2343
18  HUPA0022P        14          4023
19  HUPA0023P        14          3919
20  HUPA0024P        11          2902
21  HUPA0025P        15          4006
22  HUPA0026P        15          4320
23  HUPA0027P        16          4320
24  HUPA0028P        16          4320


In [675]:
df = df.sort_values(['patient_id', 'time'])

In [676]:
# Compute difference in minutes between consecutive rows per patient
df['time_diff'] = df.groupby('patient_id')['time'].diff().dt.total_seconds() / 60
df

,time,glucose,calories,heart_rate,steps,basal_rate,bolus_volume_delivered,carb_input,patient_id,glucose_range_level,calories_categories,date,time_diff
76147,2018-06-13 18:40:00,332.00,6.36,82.32,34.0,0.091667,0.0,0.0,HUPA0001P,Above Range,Light Activity,2018-06-13,NaN
76148,2018-06-13 18:45:00,326.00,7.73,83.74,0.0,0.091667,0.0,0.0,HUPA0001P,Above Range,Light Activity,2018-06-13,5.0
76149,2018-06-13 18:50:00,330.00,4.75,80.53,0.0,0.091667,0.0,0.0,HUPA0001P,Above Range,Resting,2018-06-13,5.0
76150,2018-06-13 18:55:00,324.00,6.36,89.13,20.0,0.091667,0.0,0.0,HUPA0001P,Above Range,Light Activity,2018-06-13,5.0
76151,2018-06-13 19:00:00,306.00,5.15,92.50,0.0,0.075000,0.0,0.0,HUPA0001P,Above Range,Light Activity,2018-06-13,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52197,2022-05-18 11:55:00,109.33,10.79,104.17,0.0,0.000000,0.0,0.0,HUPA0028P,In Range,Light Activity,2022-05-18,5.0
52198,2022-05-18 12:00:00,114.00,9.80,103.44,0.0,0.000000,0.0,0.0,HUPA0028P,In Range,Light Activity,2022-05-18,5.0
52199,2022-05-18 12:05:00,118.67,5.67,95.54,0.0,0.000000,0.0,0.0,HUPA0028P,In Range,Light Activity,2022-05-18,5.0
52200,2022-05-18 12:10:00,123.33,5.58,91.38,0.0,0.000000,0.0,0.0,HUPA0028P,In Range,Light Activity,2022-05-18,5.0


In [677]:
# Find rows where interval is not 5 minutes
irregular_intervals = df[df['time_diff'] != 5]

print("Number of irregular intervals:", irregular_intervals.shape[0])
print(irregular_intervals[['patient_id', 'time', 'time_diff']])

Number of irregular intervals: 25
      patient_id                time  time_diff
76147  HUPA0001P 2018-06-13 18:40:00        NaN
15192  HUPA0002P 2018-06-13 22:45:00        NaN
11422  HUPA0003P 2018-06-13 21:40:00        NaN
44698  HUPA0004P 2018-07-09 15:30:00        NaN
30120  HUPA0005P 2018-07-09 14:20:00        NaN
71514  HUPA0006P 2018-07-09 15:45:00        NaN
52202  HUPA0007P 2018-09-19 14:25:00        NaN
37984  HUPA0009P 2018-09-19 14:35:00        NaN
68538  HUPA0010P 2018-11-07 15:35:00        NaN
56059  HUPA0011P 2018-11-07 15:55:00        NaN
18373  HUPA0014P 2018-11-07 16:45:00        NaN
7630   HUPA0015P 2019-03-27 16:15:00        NaN
83842  HUPA0016P 2019-03-27 16:15:00        NaN
80243  HUPA0017P 2019-03-28 12:00:00        NaN
22202  HUPA0018P 2019-07-03 11:15:00        NaN
3919   HUPA0019P 2019-07-03 14:00:00        NaN
87677  HUPA0020P 2019-07-03 20:15:00        NaN
73804  HUPA0021P 2019-07-03 12:25:00        NaN
26097  HUPA0022P 2020-01-17 00:35:00        NaN
0     